## Импорт библиотек

In [ ]:
# !pip install git+https://github.com/obss/sahi

# !git clone https://github.com/pytorch/vision.git
# !cd vision
# !cp references/detection/engine.py ../

In [ ]:
from engine import train_one_epoch, evaluate

import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2

import torch
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from sahi.model import TorchVisionDetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
from IPython import display

---

## Загрузка датасетов 

In [ ]:
class PolarBearsDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "bears"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "bears", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        mask = np.array(mask)
        # mask[mask != 0] = 1
        

        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]
        

        # split the color-encoded mask into a set
        # of binary masks
        # masks = mask == obj_ids[:]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            # masks = mask == obj_ids[i]

            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            # if any([i % 1 != 0 for i in [xmin, ymin, xmax, ymax]]):
                # print(boxes)
            # if xmin == xmax or ymin == ymax:
            #     print([xmin, ymin, xmax, ymax])
                # continue

            boxes.append([xmin, ymin, xmax, ymax])

        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # boxes = torch.as_tensor(boxes, dtype=torch.int64)
        
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            # img, target = self.transforms(img, target)
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

## Mask R-CNN

In [ ]:
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    # if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        # transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def collate_fn(batch):
    return tuple(zip(*batch))

#### Обучение

In [ ]:
# use our dataset and defined transformations
dataset = PolarBearsDataset('CroppedBearsDataset/', get_transform(train=True))
dataset_test = PolarBearsDataset('CroppedBearsDataset/', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
# let's train it for 10 epochs

num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

---

#### Тест

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
transform = get_transform(False)
# test_img = Image.open('testing_ultra_set/183.png')
test_img = Image.open('PolarBearsDataset/arctic_with_bears/bears/14.JPG')
test_img = transform(test_img)

model.eval()
with torch.no_grad():
    prediction = model([test_img.to(device)])


In [ ]:
Image.fromarray(test_img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
prediction[0]['masks'].sum()

In [ ]:
prediction[0]['masks'].shape

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

---

In [ ]:
# pick one image from the test set
img, _ = dataset_test[8]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
prediction[0]['masks'].sum()

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
prediction[0]

In [ ]:
Image.fromarray(prediction[0]['masks'][2, 0].mul(255).byte().cpu().numpy())

---

#### Сохранение модели

In [ ]:
# Specify a path
model_path = "polar_bear_maskrcnn_resnet50_fpn_v1.pt"

# Save
torch.save(model, model_path)

---

#### Загрузка модели

In [ ]:
path = "polar_bear_maskrcnn_resnet50_fpn_v1.pt"

model = torch.load(path)
model.eval()

---

## Faster R-CNN

In [ ]:
def get_instance_detection_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    # if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        # transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def collate_fn(batch):
    return tuple(zip(*batch))

## Обучение

In [ ]:
# use our dataset and defined transformations
dataset = PolarBearsDataset('CroppedBearsDataset/', get_transform(train=True))
dataset_test = PolarBearsDataset('CroppedBearsDataset/', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_detection_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
# let's train it for 10 epochs

num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

---

## Тест

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
transform = get_transform(False)
# test_img = Image.open('testing_ultra_set/183.png')
test_img = Image.open('PolarBearsDataset/arctic_with_bears/bears/14.JPG')
test_img = transform(test_img)

model.eval()
with torch.no_grad():
    prediction = model([test_img.to(device)])


In [ ]:
Image.fromarray(test_img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
prediction[0]['masks'].sum()

In [ ]:
prediction[0]['masks'].shape

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

---

In [ ]:
# pick one image from the test set
img, _ = dataset_test[8]

# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
img

In [ ]:
prediction[0]['boxes'].to(torch.uint8)

In [ ]:
from torchvision.utils import draw_bounding_boxes

img = img.to(torch.uint8)

for box in prediction[0]['boxes']:
    img = draw_bounding_boxes(img, box, width=5,
                          colors="green", 
                          fill=True)

img = torchvision.transforms.ToPILImage()(img)
  
# display output
img.show()

In [ ]:
prediction[0]

In [ ]:
# сформируем функцию для показа картинки, так будет удобнее
def show_img(img, resize=(512, 512)):
    # переводим тип массива в unsigned int 8
    # этот тип может представлять числа в диапазоне (0, 255), 
    # поэтому он идеален для представления картинок
    img = img.astype("uint8")
    
    # если понадобится, то можем изменить размер картинки
    if resize:
        img = cv2.resize(img, resize)

    # составляем картинку из массива и возвращаем её
    # если вызвать эту функции в конце ячейки, то изображении автоматически покажется в ноутбуке
    return Image.fromarray(img)

In [ ]:
prediction[0]['boxes'][0][0]

In [ ]:
import cv2

In [ ]:
img = img.mul(255).permute(1, 2, 0).byte().numpy()
img = img.astype('uint8')

In [ ]:
for box in prediction[0]['boxes']:
    box = box.cpu().numpy().astype(int)
    img = cv2.rectangle(img, (box[0], box[1]),
                             (box[2], box[3]),
                             (255, 0, 0), 1)
   

show_img(img, resize=False)

In [ ]:
Image.fromarray(prediction[0]['boxes'][0, 0].mul(255).byte().cpu().numpy())

---

## Сохранение модели

In [ ]:
# Specify a path
model_path = "polar_bear_fasterrcnn_resnet50_fpn_v1.pt"

# Save
torch.save(model, model_path)

---

## Загрузка модели

In [ ]:
path = "polar_bear_fasterrcnn_resnet50_fpn_v1.pt"

model = torch.load(path)
model.eval()

## SAHI

### Тест

In [ ]:
detection_model = TorchVisionDetectionModel(
    model=model,
    confidence_threshold=0.99,
    image_size=800,
    # device="cpu",
    device="cuda:0",
    load_at_init=True,
)

#### Без SAHI

In [ ]:
# result = get_prediction("CroppedBearsDataset/bears/0.png", detection_model)
result = get_prediction("PolarBearsDataset/arctic_with_bears/bears/1.JPG", detection_model)
# result = get_prediction(read_image("CroppedBearsDataset/bears/0.png"), detection_model)

In [ ]:
result.export_visuals(export_dir="demo_data/")

display.Image("demo_data/prediction_visual.png")

#### Со слайсингом

In [ ]:
im = "PolarBearsDataset/arctic_with_bears/bears/5.jpg"
# im = "unknown1.png"

In [ ]:
result = get_sliced_prediction(
    im,
    detection_model,
    slice_height = 800,
    slice_width = 800,
    overlap_height_ratio = 0.1,
    overlap_width_ratio = 0.1,
)

In [ ]:
result.export_visuals(export_dir="demo_data/")

display.Image("demo_data/prediction_visual.png")

### Обработка нескольких фотографий одновременно

In [ ]:
model_type = "torchvision"
model_device = 'cuda:0'
# model_device = "cpu"
model_confidence_threshold = 0.99
model_input_size = 800

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detection_model = TorchVisionDetectionModel(
    model=model,
    confidence_threshold=model_confidence_threshold,
    image_size=model_input_size,
    device=model_device, 
    load_at_init=True,
)

slice_height = 800
slice_width = 800
overlap_height_ratio = 0.1
overlap_width_ratio = 0.1

# source_image_dir = "PolarBearsDataset/arctic_with_bears/bears/"
source_image_dir = "check"

In [ ]:
predict(
    model_type=model_type,
    detection_model=detection_model,
    source=source_image_dir,
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=overlap_height_ratio,
    overlap_width_ratio=overlap_width_ratio,
    project='test',
    name='1',
    export_pickle=True,
    # novisual=True
)

2 медведя не были найдены, 1 медведь найден с уверенностью 0.99.

In [ ]:
path = 'test/1/pickles/'
items = len(os.listdir(path))

for i in range(items):
    obj = pd.read_pickle(path+str(i)+'.pickle')
    bboxes = [i.to_coco_annotation().bbox for i in obj]


In [ ]:
bboxes

---

## Связка с интерфейсом

In [ ]:
# сформируем функцию для показа картинки, так будет удобнее
def show_img(img, resize=(512, 512)):
    # переводим тип массива в unsigned int 8
    # этот тип может представлять числа в диапазоне (0, 255), 
    # поэтому он идеален для представления картинок
    img = img.astype("uint8")
    
    # если понадобится, то можем изменить размер картинки
    if resize:
        img = cv2.resize(img, resize)

    # составляем картинку из массива и возвращаем её
    # если вызвать эту функции в конце ячейки, то изображении автоматически покажется в ноутбуке
    return Image.fromarray(img)

In [ ]:
def draw_bbox(img, bbox, color=(255, 0, 0), thickness=10):
    # img = img.copy()
    return cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), color=color, thickness=thickness)

In [ ]:
model_type = "torchvision"
model_device = 'cuda:0'
# model_device = "cpu"
model_confidence_threshold = 0.99
model_input_size = 800

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detection_model = TorchVisionDetectionModel(
    model=model,
    confidence_threshold=model_confidence_threshold,
    image_size=model_input_size,
    device=model_device, 
    load_at_init=True,
)

slice_height = 800
slice_width = 800
overlap_height_ratio = 0.1
overlap_width_ratio = 0.1

# source_image_dir = "PolarBearsDataset/arctic_with_bears/bears/"
# source_image_dir = "check"

In [ ]:
predict(
    model_type=model_type,
    detection_model=detection_model,
    source='PolarBearsDataset/AppTest',
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=overlap_height_ratio,
    overlap_width_ratio=overlap_width_ratio,
    project='AppTest',
    name='first',
    export_pickle=True,
    # novisual=True
)

In [ ]:
path = 'AppTest/first/pickles/'
items = len(os.listdir(path))

# bboxes_list = []

for i in range(items):
    # img = Image.open('PolarBearsDataset/AppTest/'+str(i)+'.jpg')
    img = cv2.imread('PolarBearsDataset/AppTest/'+str(i)+'.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    obj = pd.read_pickle(path+str(i)+'.pickle')
    bboxes = [i.to_coco_annotation().bbox for i in obj]
    if bboxes:
        for bb in bboxes:
            draw_bbox(img, bb)
        Image.fromarray(img).save('output/' + str(i) + '.png')
    

In [ ]:
Image('PolarBearsDataset/AppTest/8.jpg')

In [ ]:
img = cv2.imread('PolarBearsDataset/AppTest/8.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
show_img(draw_bbox(img, bboxes_list[0][0]), False)

In [ ]:
bboxes_list

---

## Сабмит

In [ ]:
# path = 'IMG'

# for name in os.listdir(path):
#     if name.endswith(".JPG"):
#         os.rename(path + '/' + name, path + '/' + name.split('.')[0] + ".jpg")

In [ ]:
model_type = "torchvision"
model_device = 'cuda:0'
# model_device = "cpu"
model_confidence_threshold = 0.99
model_input_size = 800

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detection_model = TorchVisionDetectionModel(
    model=model,
    confidence_threshold=model_confidence_threshold,
    image_size=model_input_size,
    device=model_device, 
    load_at_init=True,
)

slice_height = 800
slice_width = 800
overlap_height_ratio = 0.1
overlap_width_ratio = 0.1

# source_image_dir = "PolarBearsDataset/arctic_with_bears/bears/"
# source_image_dir = "check"
source_image_dir = 'IMG/'

In [ ]:
predict(
    model_type=model_type,
    detection_model=detection_model,
    source=source_image_dir,
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=overlap_height_ratio,
    overlap_width_ratio=overlap_width_ratio,
    project='submit',
    name='test1',
    export_pickle=True,
    # novisual=True
)

In [ ]:
def find_centres(bbxs, num):
    names = []
    x_list = []
    y_list = []
    for bb in bbxs:
        x, y = (bb[0] - bb[2] // 2), (bb[1] - bb[3] // 2)
        x_list.append(x)
        y_list.append(y)
        names.append(f"{num}.JPG")

    return names, x_list, y_list

In [ ]:
len(names)

In [ ]:
y_list

In [ ]:
names = []
x_list = []
y_list = []

path = 'submit/test1/pickles/'
items = len(os.listdir(path))

for i in range(items):
    obj = pd.read_pickle(path+str(i)+'.pickle')
    bboxes = [i.to_coco_annotation().bbox for i in obj][:3]
    
    nam, x, y = find_centres(bboxes, i)

    names += nam
    x_list += x
    y_list += y


In [ ]:
submit = pd.DataFrame(dict(name=names, x=x_list, y=y_list))

In [ ]:
submit.to_csv("deviants.csv", index=False)

In [ ]:
submit